# Examen ETL: SPARK 09/02

Se podrá utilizar toda la información que se encuentra en el campus. 

El fichero de datos sobre el que se trabajará es el de DataSetPartidos.txt.

A cada una de las preguntas hay que responder explicando brevemente que se pretende hacer antes de lanzar el código.

Al documento lo llamareís con vuestro nombre y apellido. Debeís enviarlo a mi correo de CUNEF antes del final del examen o en su defecto por email.

El lenguaje para trabajar con Spark podrá ser python o R indistintamente.

## Primera pregunta: ¿ En que se diferencian las acciones de las transformaciones en SPARK? ¿Podrías ponerme un par de ejemplo de cada una de ellas y describir que hacen, para que sirve?

## Segunda tarea: Inicializar spark context y cargar los datos desde el fichero.

In [16]:
from pyspark import SparkContext
sc = SparkContext("local", "First App")

In [17]:
data_file = "./DataSetPartidos.txt"  # direccion en el disco duro 
partidos = sc.textFile(data_file)
partidos = partidos.map(lambda x: x.split("::"))

## Tercera pregunta: Calcular el número total de goles que ha marcado el Real Sporting de Gijón.

In [18]:
partidos.take(2)

[['1',
  '1970-71',
  '1',
  'Athletic Club',
  'Barcelona',
  '1',
  '1',
  '12/09/1970',
  '21942000.0'],
 ['2',
  '1970-71',
  '1',
  'Las Palmas',
  'Atletico de Madrid',
  '1',
  '1',
  '12/09/1970',
  '21942000.0']]

In [19]:
gijon_local = partidos.filter(lambda x: ('Gijon' in x[3])).map(lambda x: (x[3], int(x[5])))
gijon_local.take(5)

[('Sporting de Gijon', 3),
 ('Sporting de Gijon', 0),
 ('Sporting de Gijon', 0),
 ('Sporting de Gijon', 3),
 ('Sporting de Gijon', 3)]

In [20]:
gijon_visit = partidos.filter(lambda x: ('Gijon' in x[4])).map(lambda x: (x[4], int(x[6])))
gijon_visit.take(5)

[('Sporting de Gijon', 0),
 ('Sporting de Gijon', 0),
 ('Sporting de Gijon', 1),
 ('Sporting de Gijon', 1),
 ('Sporting de Gijon', 1)]

In [21]:
gijon_local.map(lambda x: x[1]).sum() + gijon_visit.map(lambda x: x[1]).sum()

2042

## Cuarta pregunta: ¿ En que temporada se marcaron más goles?

In [22]:
mas_goles = partidos.map(lambda x: (x[1], (int(x[5]) + int(x[6]))))
mas_goles_key = mas_goles.reduceByKey(lambda x, y: x + y)
mas_goles_key.max(lambda x: x[1])

('2012-13', 2294)

## Quinta pregunta: ¿Cúal es el equipo y tiene el record de más goles como local? ¿ y cómo visitante?

In [23]:
mejor_equipo_local = partidos.map(lambda x: (x[3], int(x[5]))).reduceByKey(lambda x, y: x + y)
mejor_equipo_local.max(lambda x: x[1])

('Real Madrid', 2054)

In [24]:
mejor_equipo_visit = partidos.map(lambda x: (x[4], int(x[6]))).reduceByKey(lambda x, y: x + y)
mejor_equipo_visit.max(lambda x: x[1])

('Real Madrid', 1296)

## Sexta pregunta: ¿Cúales son las 3 décadas en las que más goles se metieron?

In [25]:
def get_decade(x):
    return x[2] + "0s"

In [26]:
mas_goles_decada = partidos.map(lambda x: (get_decade(x[1]), (int(x[5]) + int(x[6])))).reduceByKey(lambda x, y: x + y)
mas_goles_decada.sortBy(lambda x: x[1], ascending = False).collect()

[('00s', 20759),
 ('90s', 19426),
 ('80s', 17351),
 ('70s', 16142),
 ('10s', 10788)]

##  Séptima pregunta: ¿Cúal es la media de victorias de los equipos que han estado menos de 10 temporadas en primera división?

In [31]:
prueba = partidos.map(lambda x: ((x[3], x[1]),1)).reduceByKey(lambda x, y: x + y)
prueba2 = prueba.map(lambda x: (x[0][0], (x[0][1], x[1])))
prueba3 = prueba2.map(lambda x: (x[0], 1)).reduceByKey(lambda x, y: x + y)
prueba3.take(3)

[('Athletic Club', 45), ('Almeria', 15), ('Orihuela', 1)]

In [40]:
prueba4 = prueba3.filter(lambda x: x[1] < 10)
victorias1 = partidos.filter(lambda x: x[5] > x[6]).map(lambda x: (x[3], 1)).reduceByKey(lambda x, y: x + y)
victorias2 = partidos.filter(lambda x: x[6] > x[5]).map(lambda x: (x[4], 1)).reduceByKey(lambda x, y: x + y)

In [41]:
total_victorias = victorias1.join(victorias2)
total_victorias = total_victorias.map(lambda x: (x[0], x[1][0] + x[1][1]))
total_victorias.take(5)

[('Athletic Club', 658),
 ('Almeria', 201),
 ('Orihuela', 12),
 ('Elche', 529),
 ('CD Malaga', 276)]

In [42]:
total_partidos = partidos.map(lambda x: (x[3], 1)).reduceByKey(lambda x, y: x + y)
total = total_victorias.join(total_partidos)
total = total.map(lambda x: (x[0],(x[1][0]/x[1][1])))
total.take(5)

[('Athletic Club', 0.7966101694915254),
 ('Almeria', 0.6767676767676768),
 ('Orihuela', 0.631578947368421),
 ('Moscardo', 0.3157894736842105),
 ('Palencia CF', 0.631578947368421)]

In [43]:
final = prueba4.join(total)
final.map(lambda x: (x[0], round((x[1][1]), 2))).collect()

[('Sant Andreu', 0.71),
 ('Orihuela', 0.63),
 ('Malaga B', 0.49),
 ('Palamos', 0.58),
 ('Lorca', 0.53),
 ('Pontevedra', 0.6),
 ('Alcorcon', 0.83),
 ('Terrassa', 0.59),
 ('Real Union', 0.57),
 ('Ourense', 0.47),
 ('Langreo', 0.58),
 ('Mollerussa', 0.16),
 ('Sestao', 0.66),
 ('Calvo Sotelo', 0.59),
 ('Ciudad de Murcia', 0.7),
 ('Palencia CF', 0.63),
 ('FC Cartagena', 0.68),
 ('AD Almeria', 0.61),
 ('Alcoyano', 0.43),
 ('Mirandes', 0.63),
 ('Guadalajara', 0.62),
 ('Getafe Deportivo', 0.61),
 ('Girona', 0.72),
 ('Granada 74', 0.48),
 ('Algeciras', 0.46),
 ('Racing Ferrol', 0.55),
 ('Alzira', 0.47),
 ('Ejido', 0.61),
 ('Barakaldo', 0.68),
 ('Aviles', 0.5),
 ('Mestalla', 0.45),
 ('Linares CF', 0.56),
 ('Villarreal B', 0.71),
 ('Lorca Deportiva', 0.67),
 ('Real Burgos', 0.56),
 ('Alicante', 0.38),
 ('Llagostera', 0.71),
 ('Toledo', 0.73),
 ('Jaen', 0.59),
 ('Atletico Marbella', 0.58),
 ('Cultural Leonesa', 0.6),
 ('Moscardo', 0.32),
 ('Huesca', 0.6),
 ('Merida', 0.76),
 ('Cartagena', 0.54),
 

##  Octava pregunta: ¿Cúal es la media de goles como visitante por partido?

In [46]:
gol_visit = partidos.map(lambda x: int(x[6])).sum()
partidos2 = partidos.map(lambda x: x[6]).count()
round((gol_visit/ partidos2),2)

0.93

In [15]:
sc.stop()